In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
# se utiliza para codificar variables categóricas en variables numéricas
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Cargar los datos desde el archivo CSV
data = pd.read_csv('/content/loan_approval_dataset.csv', header=0, skipinitialspace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4269 entries, 0 to 4268
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   loan_id                   4269 non-null   int64 
 1   no_of_dependents          4269 non-null   int64 
 2   education                 4269 non-null   object
 3   self_employed             4269 non-null   object
 4   income_annum              4269 non-null   int64 
 5   loan_amount               4269 non-null   int64 
 6   loan_term                 4269 non-null   int64 
 7   cibil_score               4269 non-null   int64 
 8   residential_assets_value  4269 non-null   int64 
 9   commercial_assets_value   4269 non-null   int64 
 10  luxury_assets_value       4269 non-null   int64 
 11  bank_asset_value          4269 non-null   int64 
 12  loan_status               4269 non-null   object
dtypes: int64(10), object(3)
memory usage: 433.7+ KB


In [ ]:
# Eliminar columnas irrelevantes para la regresion
columna_eliminar = ['loan_id', 'no_of_dependents']
data.drop(columna_eliminar, axis=1, inplace=True)

# Muestra el DataFrame resultante
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4269 entries, 0 to 4268
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   education                 4269 non-null   object
 1   self_employed             4269 non-null   object
 2   income_annum              4269 non-null   int64 
 3   loan_amount               4269 non-null   int64 
 4   loan_term                 4269 non-null   int64 
 5   cibil_score               4269 non-null   int64 
 6   residential_assets_value  4269 non-null   int64 
 7   commercial_assets_value   4269 non-null   int64 
 8   luxury_assets_value       4269 non-null   int64 
 9   bank_asset_value          4269 non-null   int64 
 10  loan_status               4269 non-null   object
dtypes: int64(8), object(3)
memory usage: 367.0+ KB


In [ ]:
columnas_categoricas = data.select_dtypes(include=['object']).columns

#Procesamiento de frases o palabras
for columna in columnas_categoricas:
  # se crea una instancia (se utiliza para convetir categoricos a numericos)
  le = LabelEncoder()
  # cambia de categoricos a numericos unico
  data[columna] = le.fit_transform(data[columna])

In [ ]:
X = data.drop('loan_status', axis=1).values
y = data['loan_status'].values

# Convertir los datos en tensores de PyTorch
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [ ]:
# Definir el modelo del perceptrón
class Perceptron(nn.Module):
    def __init__(self, input_size):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_size, 1)

    def forward(self, x):
        x = torch.sigmoid(self.fc(x))
        return x

In [ ]:
# Instanciar el modelo del perceptrón
input_size = X.shape[1]
model = Perceptron(input_size)

# Definir la función de pérdida y el optimizador
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# Entrenamiento del modelo
num_epochs = 1000
for epoch in range(num_epochs):
    # Calcular las salidas del modelo
    outputs = model(X)

    # Calcular la pérdida
    loss = criterion(outputs, y.view(-1, 1))

    # Retropropagar y optimizar
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Imprimir el progreso del entrenamiento
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.3778
Epoch [200/1000], Loss: 0.3778
Epoch [300/1000], Loss: 0.3778
Epoch [400/1000], Loss: 0.3778
Epoch [500/1000], Loss: 0.3778
Epoch [600/1000], Loss: 0.3778
Epoch [700/1000], Loss: 0.3778
Epoch [800/1000], Loss: 0.3778
Epoch [900/1000], Loss: 0.3778
Epoch [1000/1000], Loss: 0.3778


In [ ]:
fila_deseada = 0

if fila_deseada < len(data):
  fila_completa = data.iloc[fila_deseada].tolist()
  print(fila_completa)
else:
  print("La fila no existe")

[0, 0, 9600000, 29900000, 12, 778, 2400000, 17600000, 22700000, 8000000, 0]


In [ ]:
X_input = torch.tensor([0, 0, 9600000, 29900000, 12, 778, 2400000, 17600000, 22700000, 8000000], dtype=torch.float32)

# Calcular la media y la desviación estándar de las características en los datos de entrenamiento
mean_train = X.mean(dim=0)
std_train = X.std(dim=0)

# Normalizar X_input utilizando la media y la desviación estándar de los datos de entrenamiento
X_input_normalized = (X_input - mean_train) / std_train

# Hacer la predicción con los datos normalizados
model.eval()  # Activar el modo de evaluación
with torch.no_grad():
    y_pred = model(X_input_normalized)

print(f'Predicción: {y_pred.item()}')

Predicción: 0.3215395510196686
